# Defects

# NEEDS UPDATING! 

# LIKELY TO BE SPLIT INTO SEPARATE NOTEBOOKS!

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)

# http://www.numpy.org/
import numpy as np

from IPython.core.display import display, HTML

# https://pandas.pydata.org/
import pandas as pd

from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.3


## 0. Access database 

### Load database

In [2]:
database = iprPy.load_database('demo')

### Load unique crystals

In [3]:
unique_crystals = pd.read_csv('unique_crystals.csv')
print(len(unique_crystals))

44


### Specify prototype-composition delimiters

In [4]:
proto_comps = {}
proto_comps['A1--Cu--fcc'] = ['Ne', 'Al', 'AlS', 'Ar', 'Ca', 'Fe', 'FeS', 'Ni', 'Cu', 'CuS', 
                              'Kr', 'Sr', 'Rh', 'Pd', 'Ag', 'Xe', 'Ir', 'Pt', 'Au', 'Pb', 
                              'Ac', 'Ce', 'Yb', 'Th']
proto_comps['A2--W--bcc'] = ['Li', 'Na', 'K', 'V', 'Cr', 'Fe', 'FeS', 'Rb', 'Nb', 'Mo', 
                             'Cs', 'Ba', 'Ta', 'W', 'Eu']
proto_comps['A3--Mg--hcp'] = ['H', 'He', 'Be', 'Mg', 'MgS', 'Sc', 'Ti', 'Co', 'Zn', 'Y', 
                              'Zr', 'Tc', 'Ru', 'Cd', 'Hf', 'Re', 'Os' 'Tl', 'Gd', 'Tb', 
                              'Dy', 'Ho', 'Er', 'Tm', 'Lu']
proto_comps['A4--C--dc'] = ['C', 'Si', 'SiS', 'Ge', 'Sn']

### Find matching parents from unique crystals

In [5]:
proto_parents = {}
for prototype in proto_comps:
    proto_parents[prototype] = unique_crystals[(unique_crystals.prototype == prototype)
                                              &(unique_crystals.composition.isin(proto_comps[prototype]))].calc_key.tolist()

In [8]:
for prototype in proto_parents:
    print(len(proto_parents[prototype]), prototype)

2 A1--Cu--fcc
0 A2--W--bcc
0 A3--Mg--hcp
0 A4--C--dc


## 1. surface_energy_static calculation

In [6]:
calculation = iprPy.load_calculation('surface_energy_static')
run_directory = iprPy.load_run_directory('demo_1')

### Build input_dict

In [7]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'surface_file', 'surface_content', 'surface_family', 'surface_cutboxvector', 'energytolerance', 'forcetolerance', 'maxiterations', 'maxevaluations', 'maxatommotion']


In [8]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build defect records
buildcombos                 defect surface_file
defect_record               free_surface

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   3 3 7

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

In [9]:
# Parse input script
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)

input_dict = DM(input_dict)
for prototype in proto_parents:
    if len(proto_parents[prototype]) > 0:
        input_dict.append('buildcombos', 'atomicarchive load_file ' + prototype)
        input_dict[prototype + '_load_key'] = 'system-info'
        input_dict[prototype + '_strainrange'] = '1e-7'
        for parent in proto_parents[prototype]:
            input_dict.append(prototype + '_load_file', parent + '.json')

In [10]:
input_dict

DataModelDict([('lammps_command', 'lmp_mpi'),
               ('buildcombos',
                ['defect surface_file',
                 'atomicarchive load_file A1--Cu--fcc']),
               ('defect_record', 'free_surface'),
               ('sizemults', '3 3 7'),
               ('A1--Cu--fcc_load_key', 'system-info'),
               ('A1--Cu--fcc_strainrange', '1e-7'),
               ('A1--Cu--fcc_load_file',
                ['3ba5fd68-696a-4543-944b-793db7b31f2d.json',
                 '37e23240-e890-41ac-b103-f716235b586f.json'])])

In [7]:
input_dict = {}

# Commands and executables
input_dict['lammps_command'] = 'lmp_mpi'
#input_dict['mpi_command'] =

# Build load information based parent calculations using family-symbols sets
input_dict['buildcombos'] = []
for family in symbols:
    input_dict['buildcombos'].append('atomicarchive2 load_file ' + family)
    input_dict[family + '_record'] = 'calculation_elastic_constants_static'
    input_dict[family + '_load_key'] = 'system-info'
    input_dict[family + '_strainrange'] = '1e-8'
    input_dict[family + '_family'] = family
    input_dict[family + '_symbols'] = symbols[family]

# Build defect information from reference files
input_dict['buildcombos'].append('defect surface_file')
input_dict['defect_record'] = 'free_surface'

# System manipulations
#input_dict['a_uvw'] =                   
#input_dict['b_uvw'] =                     
#input_dict['c_uvw'] =                     
#input_dict['atomshift'] =                  
input_dict['sizemults'] = '3 3 7'

# Units that input/output values are in
#input_dict['length_unit'] =                
#input_dict['pressure_unit'] =              
#input_dict['energy_unit'] =            
#input_dict['force_unit'] =        

# Run parameters
#input_dict['energytolerance'] = 
#input_dict['forcetolerance'] = 
#input_dict['maxiterations'] = 
#input_dict['maxevaluations'] = 
#input_dict['maxatommotion'] = 

### Prepare calculations

In [11]:
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 26 of style calculation_surface_energy_static
 - 0 are complete
 - 26 still to run
 - 0 issued errors


### Run calculations

In [12]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 10812
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 26 of style calculation_surface_energy_static
 - 24 are complete
 - 0 still to run
 - 2 issued errors


### Show results

In [13]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

b'Traceback (most recent call last):
un.py", line 110, in run
    output = sp.check_output(mpi_command + lammps_command + extra + [\'-in\'] + script_name)
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\Anaconda3\\lib\\subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\Anaconda3\\lib\\subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command \'[\'lmp_mpi\', \'-in\', \'min.in\']\' returned non-zero exit status 1.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "calc_surface_energy_static.py", line 355, in <module>
    main(*sys.argv[1:])
  File "calc_surface_energy_static.py", line 52, in main
    cutboxvector = input_dict[\'surface_cutboxvector\'])
  File "calc_surface_energy_static.py", line 159, in surface_energy
    maxiter=maxiter, maxeval=maxeval, dmax=dmax)
  File "calc_surface_energy_static.py", line 2

## 2. Calculation analysis

In [26]:
results = database.get_records_df(style=calculation.record_style)

In [21]:
# Compute surface energy
surface_energy_units = 'mJ/m^2'
results['γfs ('+surface_energy_units+')'] = uc.get_in_units(results.gamma_fs, surface_energy_units)
iprPy.analysis.assign_composition(results, database)

### Display results

In [25]:
# Display results
potential_LAMMPS_id = '1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1'
composition = 'Cu'

display_results = results[(results.potential_LAMMPS_id == potential_LAMMPS_id)
                         &(results.composition == composition)]
display_results = display_results[['γfs ('+surface_energy_units+')', 'surface_id']].sort_values('γfs ('+surface_energy_units+')')

print('Surface energies for', composition, 'using', potential_LAMMPS_id)
display(HTML(display_results.to_html(index=False)))

Surface energies for Cu using 1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1


γfs (mJ/m^2),surface_id
1094.821774,A1--Cu--fcc--111
1204.892361,A1--Cu--fcc--100
1206.858996,A1--Cu--fcc--332
1217.839498,A1--Cu--fcc--322
1255.167196,A1--Cu--fcc--221
1276.408876,A1--Cu--fcc--211
1293.713602,A1--Cu--fcc--331
1315.650655,A1--Cu--fcc--311
1322.564328,A1--Cu--fcc--110
1340.328127,A1--Cu--fcc--321


## 2. stacking_fault_map_2D calculation

In [14]:
calculation = iprPy.load_calculation('stacking_fault_map_2D')
run_directory = iprPy.load_run_directory('master_1')

### Build input_dict

In [15]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'stackingfault_file', 'stackingfault_content', 'stackingfault_family', 'stackingfault_cutboxvector', 'stackingfault_faultpos', 'stackingfault_shiftvector1', 'stackingfault_shiftvector2', 'stackingfault_numshifts1', 'stackingfault_numshifts2', 'energytolerance', 'forcetolerance', 'maxiterations', 'maxevaluations', 'maxatommotion']


In [16]:
input_dict = {}

# Commands and executables
input_dict['lammps_command'] = 'lmp_mpi'
#input_dict['mpi_command'] =

# Build load information based parent calculations using family-symbols sets
input_dict['buildcombos'] = []
for family in symbols:
    input_dict['buildcombos'].append('atomicarchive2 load_file ' + family)
    input_dict[family + '_record'] = 'calculation_elastic_constants_static'
    input_dict[family + '_load_key'] = 'system-info'
    input_dict[family + '_strainrange'] = '1e-7'
    input_dict[family + '_family'] = family
    input_dict[family + '_symbols'] = symbols[family]

# Build defect information from reference files
input_dict['buildcombos'].append('defect stackingfault_file')
input_dict['defect_record'] = 'stacking_fault'

# System manipulations
#input_dict['a_uvw'] =                   
#input_dict['b_uvw'] =                     
#input_dict['c_uvw'] =                     
#input_dict['atomshift'] =                  
input_dict['sizemults'] = '3 3 8'

# Units that input/output values are in
#input_dict['length_unit'] =                
#input_dict['pressure_unit'] =              
#input_dict['energy_unit'] =            
#input_dict['force_unit'] =        

# Run parameters
input_dict['stackingfault_numshifts1'] = '11'
input_dict['stackingfault_numshifts2'] = '11'
#input_dict['energytolerance'] = 
#input_dict['forcetolerance'] = 
#input_dict['maxiterations'] = 
#input_dict['maxevaluations'] = 
#input_dict['maxatommotion'] = 

### Prepare calculations

In [17]:
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 1 of style calculation_stacking_fault_map_2D
 - 0 are complete
 - 1 still to run
 - 0 issued errors


### Run calculations

In [18]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 10256
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 1 of style calculation_stacking_fault_map_2D
 - 1 are complete
 - 0 still to run
 - 0 issued errors


## 3. point_defect_static calculation

In [19]:
calculation = iprPy.load_calculation('point_defect_static')
run_directory = iprPy.load_run_directory('master_1')

### Build input_dict

In [20]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'pointdefect_file', 'pointdefect_content', 'pointdefect_family', 'pointdefect_type', 'pointdefect_atype', 'pointdefect_pos', 'pointdefect_dumbbell_vect', 'pointdefect_scale', 'energytolerance', 'forcetolerance', 'maxiterations', 'maxevaluations', 'maxatommotion']


In [21]:
input_dict = {}

# Commands and executables
input_dict['lammps_command'] = 'lmp_mpi'
#input_dict['mpi_command'] =

# Build load information based parent calculations using family-symbols sets
input_dict['buildcombos'] = []
for family in symbols:
    input_dict['buildcombos'].append('atomicarchive2 load_file ' + family)
    input_dict[family + '_record'] = 'calculation_elastic_constants_static'
    input_dict[family + '_load_key'] = 'system-info'
    input_dict[family + '_strainrange'] = '1e-7'
    input_dict[family + '_family'] = family
    input_dict[family + '_symbols'] = symbols[family]

# Build defect information from reference files
input_dict['buildcombos'].append('defect pointdefect_file')
input_dict['defect_record'] = 'point_defect'

# System manipulations
#input_dict['a_uvw'] =                   
#input_dict['b_uvw'] =                     
#input_dict['c_uvw'] =                     
#input_dict['atomshift'] =                  
input_dict['sizemults'] = '10 10 10'

# Units that input/output values are in
#input_dict['length_unit'] =                
#input_dict['pressure_unit'] =              
#input_dict['energy_unit'] =            
#input_dict['force_unit'] =        

# Run parameters
#input_dict['energytolerance'] = 
#input_dict['forcetolerance'] = 
#input_dict['maxiterations'] = 
#input_dict['maxevaluations'] = 
#input_dict['maxatommotion'] = 

### Prepare calculations

In [22]:
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 9 of style calculation_point_defect_static
 - 0 are complete
 - 9 still to run
 - 0 issued errors


### Run calculations

In [23]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 10256
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 9 of style calculation_point_defect_static
 - 9 are complete
 - 0 still to run
 - 0 issued errors


### Show results

In [24]:
energy_units = 'eV'

In [25]:
results = database.get_records_df(style=calculation.record_style, potential_LAMMPS_id='2004--Zhou-X-W--Al--LAMMPS--ipr2')

In [26]:
results = results[results.reconfigured == False]
results['Ef ('+energy_units+')'] = uc.get_in_units(results.E_f, energy_units)
results = results[['Ef ('+energy_units+')', 'pointdefect_id']].sort_values('Ef ('+energy_units+')')
display(HTML(results.to_html(index=False)))

Ef (eV),pointdefect_id
0.676481,A1--Cu--fcc--vacancy
1.296545,A1--Cu--fcc--1nn-divacancy
1.311998,A1--Cu--fcc--2nn-divacancy
1.864239,A1--Cu--fcc--100-dumbbell
1.958664,A1--Cu--fcc--octahedral-interstitial
2.042814,A1--Cu--fcc--110-dumbbell
2.043344,A1--Cu--fcc--crowdion-interstitial
2.115240,A1--Cu--fcc--111-dumbbell
2.149543,A1--Cu--fcc--tetrahedral-interstitial
